In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/movie', './input/rating']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

## ** Análise explorátoria dos dados obtidos em ratings de filmes**
![Iesb](https://logo.clearbit.com/iesb.br)

> Stable benchmark dataset. 20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags. Released 4/2015; updated 10/2016 to update links.csv and add tag genome data.  [MovieLens 20m Dataset Oficial](https://grouplens.org/datasets/movielens/20m/)
 
 
 ## ** Objetivo**
 > Realizar análise descritiva e explorátoria dos dados 
 
 ## **Etapas**
1. Conhecendo as variáveis
1. Análise exploratória
1. Visualizando e tratando valores nulos (Data Cleaning)
1. One hot encoding (tratando tipo de dados)
1. Criando um modelo de machine learning
1. Conclusão

# Importando bibiliotecas

In [1]:
# FIRST-AUTHOR: remove plotting
# import matplotlib.pyplot as plt # Bibilioteca util para criar gráficos
# import pandas as pd # Bibilioteca para auxiliar a importar e maniular nossos dataframes
exec(os.environ['IREWR_IMPORTS'])
# FIRST-AUTHOR: remove ML code
# from sklearn.tree import DecisionTreeClassifier #responsável pela geração do modelo 
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import RandomForestClassifier
import numpy as np # Bibilioteca útil para realizar operações matemáticas
# FIRST-AUTHOR: remove plotting, ML code
# import seaborn as sns # Bibilioteca utilizada para dar um toque especial nos gráficos
# #import chardet   #Trabalha com leitura de arquivos, acredito que n será necessário utiliza=lá
# plt.style.use('ggplot') #Customização de gráficos
# plt.style.use("seaborn-white")
# import os
# from mpl_toolkits.mplot3d import Axes3D

# import random
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# FIRST-AUTHOR: remove path printing
# import os
# print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

# Informações a respeito das variáveis
[Content and Use of Files](http://files.grouplens.org/datasets/movielens/ml-20m-README.html)

# Importando os dados

In [2]:
movies = pd.read_csv('./input/movie.scaled.csv')
ratings = pd.read_csv('./input/rating.scaled.csv')

movies.sort_values(by='movieId', inplace=True)
movies.reset_index(inplace=True, drop=True)
ratings.sort_values(by='movieId', inplace=True)
ratings.reset_index(inplace=True, drop=True)

# **1. Conhecendo as variáveis**

In [3]:
#Dimensão  datasets
print("Dimensão  dataset de movies")
print("Colunas:", movies.shape[1],"\nLinhas:", movies.shape[0])
print("-")
print("Dimensão  dataset de ratings")
print("Colunas:", ratings.shape[1],"\nLinhas:", ratings.shape[0])

Dimensão  dataset de movies
Colunas: 3 
Linhas: 27278
-
Dimensão  dataset de ratings
Colunas: 4 
Linhas: 20000263


In [4]:
#Verificando os primeiros registros do conjunto de dados
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,97809,1,3.0,2008-06-11 04:47:11
1,106140,1,5.0,2013-01-29 03:33:49
2,106138,1,3.0,2002-07-31 15:48:53
3,70354,1,4.5,2011-02-13 18:55:40
4,70355,1,3.5,2008-01-26 16:56:54


In [6]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp     object
dtype: object

In [7]:
movies.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


## Calculando a simetria dos dados
> Um valor zero indica uma distribuição simétrica, um valor maior que zero ou menor indica uma distribuição assimétrica.

In [9]:
# FIRST-AUTHOR: make notebook run
# ratings.skew()
ratings.skew(numeric_only=True)

userId     0.001475
movieId    3.025376
rating    -0.655311
dtype: float64

In [10]:
# FIRST-AUTHOR: make notebook run
# movies.skew()
movies.skew(numeric_only=True)

movieId   -0.066547
dtype: float64

## **2. Visualizando e tratando valores nulos (Data Cleaning)**

In [11]:
# Dividindo o título e o ano de lançamento em colunas separadas no dataframe de filmes
#Convertertendo ano para timestamp.

movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]

In [12]:
#f,ax = plt.subplots(figsize=(10,8))
#sns.heatmap(ratings.corr(), annot=True, linewidths=.7, fmt= '.2f',ax=ax)
#plt.show()

In [13]:
#categorizando os gêneros de filmes corretamente. 
#Trabalhar mais tarde com + 20MM de linhas de strings consome muito recurso
genres_unique = pd.DataFrame(movies.genres.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre']) 
movies = movies.join(movies.genres.str.get_dummies().astype(bool))
movies.drop('genres', inplace=True, axis=1)

In [14]:
genres_unique

,genre
0,Adventure
1,Animation
2,Children
3,Comedy
4,Fantasy
5,Romance
6,Drama
7,Action
8,Crime
9,Thriller


In [15]:
movies.head()

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995.0,False,False,True,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,Jumanji,1995.0,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,Grumpier Old Men,1995.0,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
3,4,Waiting to Exhale,1995.0,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
4,5,Father of the Bride Part II,1995.0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
# Modificando o formato do registro de data e hora da avaliação
ratings.timestamp = pd.to_datetime(ratings.timestamp, infer_datetime_format=True)
ratings.timestamp = ratings.timestamp.dt.year

/tmp/ipykernel_1336073/1815208427.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  ratings.timestamp = pd.to_datetime(ratings.timestamp, infer_datetime_format=True)


In [17]:
ratings['timestamp'].unique()

array([2008, 2013, 2002, 2011, 2012, 1999, 2001, 2007, 1997, 2009, 2006,
       1998, 2010, 2014, 2003, 1996, 2000, 2005, 2004, 2015, 1995],
      dtype=int32)

## Limpando registros vazios

In [18]:
# FIRST-AUTHOR: remove plotting
# sns.heatmap(movies.isnull(),yticklabels=False,cbar=False,cmap='viridis')
# plt.show()
_ = movies.isnull()

In [19]:
#Contando valores nulos
movies.isnull().sum().sort_values(ascending=False).head(10)

year        22
movieId      0
Fantasy      0
War          0
Thriller     0
Sci-Fi       0
Romance      0
Mystery      0
Musical      0
IMAX         0
dtype: int64

In [20]:
# FIRST-AUTHOR: remove plotting
# sns.heatmap(ratings.isnull(),yticklabels=False,cbar=False,cmap='viridis')
# plt.show()
_ = ratings.isnull()

In [21]:
#contando valores nulos
ratings.isnull().sum().sort_values(ascending=False).head(10)

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [22]:
# Removendo valores nulos do datasets movies
movies.dropna(inplace=True)

In [23]:
movies.isnull().sum().sort_values(ascending=False).head(10)

movieId     0
Fantasy     0
War         0
Thriller    0
Sci-Fi      0
Romance     0
Mystery     0
Musical     0
IMAX        0
Horror      0
dtype: int64

# **3. Análise explorátoria**

# Número de filmes e classificações por ano

> Número de filmes lançados por ano aumentou  quase exponencialmente até 2009, depois diminuiu significativamente em 2014 (os dados de 2015 estão incompletos).  Nenhuma avaliação antes de 1995, provavelmente relacionada à disponibilidade da Internet para o público em geral.

In [24]:
#df_mv_year = movies.groupby('movieId')['year']

In [25]:
dftmp = movies[['movieId', 'year']].groupby('year')

# FIRST-AUTHOR: remove plotting
# fig, ax1 = plt.subplots(figsize=(15,8))
# ax1.plot(dftmp.year.first(), dftmp.movieId.nunique(), "b")
# ax1.grid(False)
_ = dftmp.year.first()
_ = dftmp.movieId.nunique()

dftmp = ratings[['rating', 'timestamp']].groupby('timestamp')
# FIRST-AUTHOR: remove plotting
# ax2 = ax1.twinx() #Plotando os dados de avaliações no eixo
# ax2.plot(dftmp.timestamp.first(), dftmp.rating.count(), "r")
# ax2.grid(False)

# ax1.set_xlabel('Ano')
# ax1.set_ylabel('Número de filmes liberados'); ax2.set_ylabel('Número de avaliações')
# plt.title('Filmes por ano')
# plt.show()
_ = dftmp.timestamp.first()
_ = dftmp.rating.count()

In [26]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
# FIRST-AUTHOR: remove plotting
#     nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
#     plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
# FIRST-AUTHOR: remove plotting
#         plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
# FIRST-AUTHOR: remove plotting
#             valueCounts.plot.bar()
#         else:
#             columnDf.hist()
#         plt.ylabel('counts')
#         plt.xticks(rotation = 90)
#         plt.title(f'{columnNames[i]} (column {i})')
#     plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
#     plt.show()

In [27]:
# Scatter and density plots
# FIRST-AUTHOR: remove unused code
# def plotScatterMatrix(df, plotSize, textSize):
#     df = df.select_dtypes(include =[np.number]) # keep only numerical columns
#     # Remove rows and columns that would lead to df being singular
#     df = df.dropna('columns')
#     df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
#     columnNames = list(df)
#     if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
#         columnNames = columnNames[:10]
#     df = df[columnNames]
#     ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
#     corrs = df.corr().values
#     for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
#         ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
#     plt.suptitle('Scatter and Density Plot')
#     plt.show()

In [28]:
#Distribuição dos dados de avaliação e ano
plotPerColumnDistribution(ratings, 10, 5)

In [29]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [30]:
#Quantidade de usuários
ratings['userId'].count()

20000263

In [31]:
ratings.groupby('timestamp')['userId'].count()

timestamp
1995          4
1996    1612609
1997     700982
1998     308070
1999    1198384
2000    1953659
2001    1186125
2002     869719
2003    1035878
2004    1170049
2005    1803158
2006    1171836
2007    1053430
2008    1158777
2009     930036
2010     903691
2011     766366
2012     731389
2013     599327
2014     562888
2015     283886
Name: userId, dtype: int64

In [32]:
#Quantidade de usuários registrados por ano
# FIRST-AUTHOR: remove plotting
# ratings.groupby('timestamp')['userId'].count().plot(figsize=(15,8), color="g")
# plt.ylabel("Qtd de usuários")
# plt.xlabel("Ano")
# plt.title("Contagem de usuários por ano")
# plt.show()
ratings.groupby('timestamp')['userId'].count()

timestamp
1995          4
1996    1612609
1997     700982
1998     308070
1999    1198384
2000    1953659
2001    1186125
2002     869719
2003    1035878
2004    1170049
2005    1803158
2006    1171836
2007    1053430
2008    1158777
2009     930036
2010     903691
2011     766366
2012     731389
2013     599327
2014     562888
2015     283886
Name: userId, dtype: int64

# Número acumulado de filmes, no total e por gênero.
> Em média, os filmes são categorizados em 2 gêneros (ou seja, o número de filmes-gêneros  dobra o número de filmes). Comédia e Drama  são os principais gêneros.

In [33]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(10,5))
dftmp = movies[['movieId', 'year']].groupby('year')
df = pd.DataFrame({'All_movies' : dftmp.movieId.nunique().cumsum()})

#Histograma para cada gênero individual
for genre in genres_unique.genre:
    dftmp = movies[movies[genre]][['movieId', 'year']].groupby('year')
    df[genre]=dftmp.movieId.nunique().cumsum()
df.fillna(method='ffill', inplace=True)
# FIRST-AUTHOR: remove plotting
# df.loc[:,df.columns!='All_movies'].plot.area(stacked=True, figsize=(15,8))
df.loc[:,df.columns!='All_movies']

# Histograma de plotagem para todos os filmes
# FIRST-AUTHOR: remove plotting
# plt.plot(df['All_movies'], marker='o', markerfacecolor='black')
# plt.xlabel('Ano')
# plt.ylabel('Acumulativo de filmes por gênero')
# plt.title('Total de filmes por gênero') # Many movies have multiple genres, so counthere is higher than number of movies
# plt.legend(loc=(1.05,0), ncol=2)
# plt.show()

# #  dispersão simples do número de filmes marcados com cada gênero
# plt.figure(figsize=(15,8))
# barlist = df.iloc[-1].sort_values().plot.bar()
# barlist.patches[0].set_color('b')
# plt.xticks(rotation='vertical')
# plt.title('Filmes por gênero')
# plt.xlabel('Gênero')
# plt.ylabel('Número de filmes')
# plt.show()
_ = df['All_movies']
_ = df.iloc[-1].sort_values()

/tmp/ipykernel_1336073/897554533.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [34]:
dftmp = ratings[['movieId','rating']].groupby('movieId').mean()

# inicializando uma lista vazia para capturar estatísticas básicas por gênero
rating_stats = []
# Histograma geral do lote de todas as classificações
# FIRST-AUTHOR: remove plotting
# dftmp.hist(bins=25, grid=False, edgecolor='b', normed=True, label ='All genres', figsize=(15,8))

# Histograma com linhas kde para melhor visibilidade por gênero
for genre in genres_unique.genre:
    dftmp = movies[movies[genre]==True]
    dftmp = ratings[ratings.set_index('movieId').index.isin(dftmp.set_index('movieId').index)]
    dftmp = dftmp[['movieId','rating']].groupby('movieId').mean()
# FIRST-AUTHOR: remove plotting
#     dftmp.rating.plot(grid=False, alpha=0.6, kind='kde', label=genre)
    dftmp.rating
    avg = dftmp.rating.mean()
    std = dftmp.rating.std()
    rating_stats.append((genre, avg, std))
# FIRST-AUTHOR: remove plotting
# plt.legend(loc=(1.05,0), ncol=2)
# plt.xlim(0,5)
# plt.xlabel('Avaliações')
# plt.ylabel('Densidade')
# plt.title('Histograma de avaliação de filmes')
# plt.show()